Spinorama plot with Altair

In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
from src.spinorama.analysis import sound_power
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker, graph_melt
from src.spinorama.graph import graph_params_default

# df = parse_graphs_speaker('Adam', 'Adam S2V', 'klippel')
df = parse_graphs_speaker('Neumann', 'Neumann KH 80', 'klippel')
# df = parse_graphs_speaker('Genelec', 'Genelec 8030A', 'princeton')
# print(df)

In [ ]:
nearest = alt.selection(
    type='single',
    nearest=True,
    on='mouseover',
    fields=['Freq'],
    empty='none')

In [ ]:
def graph_spinorama(dfu, graph_params):
    xmin = graph_params['xmin']
    xmax = graph_params['xmax']
    ymin = graph_params['ymin']
    ymax = graph_params['ymax']
    if xmax == xmin:
        logging.error('Graph configuration is incorrect: xmin==xmax')
    if ymax == ymin:
        logging.error('Graph configuration is incorrect: ymin==ymax')
    # add selectors                                                                                                                          
    selectorsMeasurements = alt.selection_multi(
        fields=['Measurements'], 
        bind='legend')
    scales = alt.selection_interval(
        bind='scales'
    )
    # main charts
    line=alt.Chart(dfu, title=f'CEA2034').mark_line(clip=True).transform_filter(
        alt.FieldOneOfPredicate(
            field='Measurements', 
            oneOf=['On Axis', 'Listening Window', 'Early Reflections', 'Sound Power'])
    ).encode(
        alt.X('Freq:Q', title='Freqency (Hz)',
              scale=alt.Scale(type='log', base=10, nice=False, domain=[xmin, xmax]), 
              axis=alt.Axis(format='s')),
        alt.Y('dB:Q', scale=alt.Scale(zero=False, domain=[ymin, ymax]), axis=alt.Axis(title='Sound Pressure (dB)')),
        alt.Color('Measurements', type='nominal', sort=None),
        opacity=alt.condition(selectorsMeasurements, alt.value(1), alt.value(0.2))
    ).properties(
        width=graph_params['width'],
        height=graph_params['height']
    )
    
    di=alt.Chart(dfu).mark_line(clip=True).transform_filter(
        alt.FieldOneOfPredicate(field='Measurements', oneOf=['Early Reflections DI', 'Sound Power DI'])
    ).encode(
        alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[xmin, xmax])),
        alt.Y('dB:Q',   scale=alt.Scale(zero=False, domain=[60, 100])),
        alt.Color('Measurements', type='nominal', sort=None),
        opacity=alt.condition(selectorsMeasurements, alt.value(1), alt.value(0.2))
    )
    circle=alt.Chart(dfu).mark_circle(size=100).encode(
        alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[xmin, xmax])),
        alt.Y('dB:Q',   scale=alt.Scale(zero=False)),
        alt.Color('Measurements', type='nominal', sort=None),
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        tooltip=['Measurements', 'Freq', 'dB']
    ).transform_calculate(Freq=f'format(datum.Freq, ".0f")', dB=f'format(datum.dB, ".1f")')
    
    # assemble elements together
    spin = (circle + (line + di) #.resolve_scale(y='independent')).resolve_scale(y='independent'
    ).add_selection(
        selectorsMeasurements
    ).add_selection(
        scales
    ).add_selection(
        nearest
    ).configure_legend(
        orient='top'
    ).configure_title(
        orient='top',
        anchor='middle',
        fontSize=18
    )
    return spin




dfu = df['CEA2034']
params = graph_params_default
params['width'] = 800
params['height'] = 500
graph_spinorama(dfu, params)

Compare computed sound power wrt the version done by Klippel
Matching is good for low frequency but error increases with frequency

In [ ]:
#df = pd.DataFrame({'Freq': [200, 2000, 10000, 12000, 16000, 20000], '10°': [2,2,2,2,2,2], '-10°': [2,2,2,2,2,2]})
sp = sound_power(df['SPL Horizontal_unmelted'], df['SPL Vertical_unmelted'])

In [ ]:
dfu = df['CEA2034_unmelted']
check = pd.DataFrame({
    'Freq': dfu.Freq, 
    'Control': dfu['Sound Power'], 
    'Computed': sp.dB})
check
mcheck = graph_melt(check)
alt.Chart(mcheck).mark_line(clip=True).encode(
    x=alt.X('Freq', scale=alt.Scale(type='log', nice=False)),
    y=alt.Y('dB', scale=alt.Scale(domain=[90, 106])),
    color=alt.Color('Measurements'))